In [ ]:
#!pip install pykml

In [ ]:
## Optimal Params for pictures: zoom = 16 , size = "1280x1280"

## Imports and Intalization

In [ ]:
import requests
# from pykml import parser
from lxml import etree
import os
from lxml.etree import ParseError
from geopy.geocoders import Nominatim
import pickle
import pandas as pd
import re
import zipfile

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import zipfile
def unzip_images(zip_path, extract_to):
    # Check if the extraction directory exists, create if not
    os.makedirs(extract_to, exist_ok=True)

    # Unzip the archive
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"Images extracted to {extract_to}")


## Parse KML File and Create intial Dataset

In [ ]:
 #%%writefile create_dataset.py

def get_map_snapshot(api_key, coordinates, zoom=16, size="1280x1280", map_type="satellite"):
    """
    Function to get a map snapshot from Google Maps Static API.

    :param api_key: Your Google Maps Static API key.
    :param coordinates: A tuple or list of the latitude and longitude (e.g., (39.6026, -9.0708)).
    :param zoom: Zoom level of the map. Default is 12.
    :param size: Size of the map image. Default is 600x300.
    :param map_type: Type of the map (roadmap, satellite, hybrid, terrain). Default is satellite.
    :return: The response content (image bytes).
    """
    base_url = "https://maps.googleapis.com/maps/api/staticmap?"

    params = {
        "center": f"{coordinates[0]},{coordinates[1]}",
        "zoom": zoom,
        "size": size,
        "maptype": map_type,
        "key": api_key
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        return response.content
    else:
        raise Exception(f"Error fetching map: {response.status_code} - {response.text}")

def save_image(image_content, directory, filename):
    """
    Function to save the image content to a file.

    :param image_content: The image content (bytes).
    :param directory: The directory to save the image.
    :param filename: The filename to save the image.
    """
    os.makedirs(directory, exist_ok=True)
    filepath = os.path.join(directory, filename)
    with open(filepath, "wb") as file:
        file.write(image_content)
    print(f"Saved image to: {filepath}")
    return os.path.basename(filepath)

def get_location_details(latitude, longitude):
    geolocator = Nominatim(user_agent="myGeoAPI")
    location = geolocator.reverse((latitude, longitude), exactly_one=True)
    return location.address if location else "Location details not found"

def parse_kml(file_path):
    """
    Function to parse a KML file using the pykml parser and extract coordinates, folder names, and placemark names.

    :param file_path: Path to the KML file.
    :return: List of tuples, each containing coordinates in decimal degrees, folder name, and placemark name.
    """
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"KML file not found: {file_path}")

    details_list = []

    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            doc = parser.parse(file).getroot()

        # Iterate over all folders in the document
        for folder in doc.Document.findall('.//{http://www.opengis.net/kml/2.2}Folder'):
            folder_name = folder.name.text if hasattr(folder, 'name') else "Unnamed Folder"

            # Iterate over all placemarks in the folder
            for placemark in folder.findall('.//{http://www.opengis.net/kml/2.2}Placemark'):
                placemark_name = placemark.name.text if hasattr(placemark, 'name') else "Unnamed Placemark"

                # Iterate over all coordinates in the placemark
                for coords in placemark.findall('.//{http://www.opengis.net/kml/2.2}coordinates'):
                    coord_text = coords.text.strip()
                    coord_parts = coord_text.split(',')
                    if len(coord_parts) >= 2:
                        lon, lat = coord_parts[0], coord_parts[1]
                        try:
                            loc_details = get_location_details(float(lat), float(lon))
                            details_list.append([(float(lat), float(lon)) , folder_name, placemark_name,loc_details])
                        except ValueError:
                            print(f"Skipping invalid coordinates: {coord_text}")
            #     continue_user = input('Continue?')
            #     if continue_user == 'No':
            #       break
            # break
    except ParseError as e:
        print(f"Error parsing the KML file: {e}")

    return details_list

def process_kml_coordinates(api_key, kml_file_path, save_directory):
    """
    Function to process a KML file, fetch map snapshots, and save them with running numbers.

    :param api_key: Your Google Maps Static API key.
    :param kml_file_path: Path to the KML file.
    :param save_directory: Directory to save the images.
    """
    details_list = parse_kml(kml_file_path)

    for i, coords in enumerate(details_list, start=1):
        try:
            image_content = get_map_snapshot(api_key, coords[0])
            save_image(image_content, save_directory, f"map_snapshot_{i}.png")
        except Exception as e:
            print(f"Error with coordinate {coords}: {e}")

def extract_country(location):
    search_res = re.search(r'[\w\s]+$', location)
    if search_res:
      if search_res.group().strip() == 'Maroc ⵍⵎⵖⵔⵉⴱ المغرب':
        country = 'Marraco'
        return country
      elif search_res.group().strip() == 'ישראל':
        country = 'Israel'
        return country
      else:
        return search_res.group().strip()
    else:
      return None  # If no country is found, return None

def add_terrain(country):
  terrain_dict = {'Sri Lanka':'Tropical', 'Maldives': 'Tropical' ,\
                  'El Salvador': 'Tropical', 'Costa Rica': 'Tropical',\
                  'Ecuador': 'Tropical','Marraco': 'Desert','Panamá':'Tropical',\
                  'Indonesia':'Tropical','Nicaragua':'Tropical'}
  return terrain_dict.get(country, 'Unknown')


def add_image_path(details_df):
  api_key = 'YOUR GOOGLE MAPS API KEY HERE'
  for index, row in details_df.iterrows():
    placemark = row['placemark_num']
    coordinates = row['coordinates']
    image_content = get_map_snapshot(api_key, coordinates)
    image_file_path = save_image(image_content, save_directory, f"map_snapshot_{placemark}.png")
    details_df.at[index, 'image_path'] = image_file_path  # Ensure the path is correctly assigned back to the DataFrame
  return details_df

# if __name__ == "__main__":
#     # Absolute path to the KML file
#     kml_file_path = r"/content/Satalite Surfing Images.kml"

#     # Absolute path to the directory where you want to save the images
#     save_directory = r"/content/Sat_Images"

#     # Your Google Maps Static API key
#     api_key = "YOUR_API_KEY_HERE"
      # data_without_photos = parse_kml(kml_file_path)
      # details_df = pd.DataFrame(data_without_photos, columns=['coordinates', 'tagger', 'placemark_num', 'location'])
      # details_df['country'] = details_df['location'].apply(extract_country)
      # details_df['terrain'] = details_df['country'].apply(add_terrain)
      # details_df_with_images = add_image_path(details_df)


## Feature Engineering

### Functions

In [1]:
def extract_country(location):
    search_res = re.search(r'[\w\s]+$', location)
    if search_res:
      if search_res.group().strip() == 'Maroc ⵍⵎⵖⵔⵉⴱ المغرب':
        country = 'Marraco'
        return country
      elif search_res.group().strip() == 'ישראל':
        country = 'Israel'
        return country
      else:
        return search_res.group().strip()
    else:
      return None  # If no country is found, return None

def add_terrain(country):
  terrain_dict = {'Sri Lanka':'Tropical', 'Maldives': 'Tropical' ,\
                  'El Salvador': 'Tropical', 'Costa Rica': 'Tropical',\
                  'Ecuador': 'Tropical','Marraco': 'Desert','Panamá':'Tropical',\
                  'Indonesia':'Tropical','Nicaragua':'Tropical'}
  return terrain_dict.get(country, 'Unknown')


def add_image_path(details_df):
  api_key = 'YOUR API KEY HERE'
  for index, row in details_df.iterrows():
    placemark = row['placemark_num']
    coordinates = row['coordinates']
    image_content = get_map_snapshot(api_key, coordinates)
    image_file_path = save_image(image_content, save_directory, f"map_snapshot_{placemark}.png")
    details_df.at[index, 'image_path'] = image_file_path  # Ensure the path is correctly assigned back to the DataFrame
  return details_df

## Creating the dataset - Main Cell

In [ ]:
data_without_photos = parse_kml(kml_file_path)
details_df = pd.DataFrame(data_without_photos, columns=['coordinates', 'tagger', 'placemark_num', 'location'])
details_df['country'] = details_df['location'].apply(extract_country)
details_df['terrain'] = details_df['country'].apply(add_terrain)
details_df_with_images = add_image_path(details_df)
# details_df_with_images.to_csv('/content/drive/MyDrive/Satalite_Surf_Project/Satalite_Surf_Image_cleaned_dataset.csv')

##Unzip Photos

In [ ]:
import zipfile
from google.colab import files

def zip_directory(folder_path, output_zip):
    """Zip the contents of an entire directory."""
    with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, os.path.join(folder_path, '..')))

# Specify the directory you want to download
folder_path = '/content/Sat_Images'  # Adjust this to your folder path
output_zip = '/content/Sat_Images.zip'  # Name for the output ZIP file

# Create a ZIP file
zip_directory(folder_path, output_zip)

# Download the ZIP file
files.download(output_zip)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Data Orginazing

This Stage was Executed after manual reviewal of the tagged data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
data_cleaned = pd.read_csv('/content/drive/MyDrive/Satalite_Surf_Project/Satalite_Surf_Image_cleaned_dataset.csv')

##Creating Final Cleaned_df

In [ ]:
## Cleaning Irrelvant columns
full_data_df = pd.read_csv('/content/drive/MyDrive/Satalite_Surf_Project/Satalite_Surf_image_data.csv')
full_data_df = full_data_df.drop(columns=['Unnamed: 0','Unnamed: 10'])

In [ ]:
# Splitting and flattening the 'extra_features' into a list of unique features
full_data_df['extra_features_list'] = full_data_df['extra features'].dropna().apply(lambda x: x.split(' + '))

# Flatten the list and get unique features
all_features = set()
full_data_df['extra_features_list'].dropna().apply(lambda features: [all_features.add(feature) for feature in features])

# Create columns for each feature and fill with binary values (0 or 1)
for feature in all_features:
    full_data_df[feature] = full_data_df['extra features'].apply(lambda x: 1 if isinstance(x, str) and feature in x else 0)

print(full_data_df)

                                 coordinates tagger placemark_num  \
0    (32.22855629832505, -116.9234467023862)  Gilad     Gilad_001   
1    (32.20238866696115, -116.9139428948928)  Gilad     Gilad_002   
2    (32.18326958506614, -116.9099017074971)  Gilad     Gilad_003   
3    (30.54467815330961, -9.726458682969735)   Alon        alon 1   
4     (29.88665941651898, -9.78552918565589)   Alon       alon 10   
..                                       ...    ...           ...   
386  (49.10637675770386, -125.8767321241412)    Ron        Ron 60   
387  (49.10891761273925, -125.8811443808606)    Ron        Ron 61   
388  (-35.95255978275283, 150.1582975453066)    Ron         Ron 7   
389  (-35.86753610565759, 150.1635278545278)    Ron         Ron 8   
390  (-35.81141215306061, 150.2261763007545)    Ron         Ron 9   

                                              location    country  terrain  \
0    Caseta de vigilancia, Calle Mar De Los Erisos,...     México  Unknown   
1    Mision de 

In [ ]:
full_data_df = full_data_df.drop(columns=['extra features'])

In [ ]:
full_data_df.to_csv('/content/drive/MyDrive/Satalite_Surf_Project/Satalite_Surf_Image_cleaned_dataset.csv')

## Get DataFrame by Feature

In [ ]:
def get_df_by_feature(df):
    # Positive examples
    bay_df = df[df['Bay'] == 1]
    rivermouth_df = df[df['Rivermouth'] == 1]
    wavepattern_df = df[df['Wave Pattern'] == 1]

    # Determine the number of positive examples
    num_bay = len(bay_df)
    num_rivermouth = len(rivermouth_df)
    num_wavepattern = len(wavepattern_df)

    # Negative examples, matched in number to positive examples
    non_bay_df = df[df['Bay'] == 0].sample(min(num_bay, len(df[df['Bay'] == 0])), random_state=42)
    non_rivermouth_df = df[df['Rivermouth'] == 0].sample(min(num_rivermouth, len(df[df['Rivermouth'] == 0])), random_state=42)
    non_wavepattern_df = df[df['Wave Pattern'] == 0].sample(min(num_wavepattern, len(df[df['Wave Pattern'] == 0])), random_state=42)

    # Append negative examples to each DataFrame
    bay_df = pd.concat([bay_df, non_bay_df]).sample(frac=1).reset_index(drop=True)  # Shuffle
    rivermouth_df = pd.concat([rivermouth_df, non_rivermouth_df]).sample(frac=1).reset_index(drop=True)  # Shuffle
    wavepattern_df = pd.concat([wavepattern_df, non_wavepattern_df]).sample(frac=1).reset_index(drop=True)  # Shuffle

    # DataFrame containing examples that do not belong to any specific class
    other_df = df[(df['Wave Pattern'] != 1) & (df['Rivermouth'] != 1) & (df['Bay'] != 1)]

    return bay_df, rivermouth_df, wavepattern_df, other_df

def get_balanced_onlygood_df(df):
    # Define positive examples: having any of the features
    onlygood_df = df[(df['Wave Pattern'] == 1) | (df['Rivermouth'] == 1) | (df['Bay'] == 1)]

    # Define negative examples: having none of the features
    non_onlygood_df = df[(df['Wave Pattern'] != 1) & (df['Rivermouth'] != 1) & (df['Bay'] != 1)]

    # Calculate the minimum number of examples to make the dataset balanced
    min_count = min(len(onlygood_df), len(non_onlygood_df))

    # Sample from both positive and negative examples to make them balanced
    onlygood_df_balanced = onlygood_df.sample(min_count, random_state=42)
    non_onlygood_df_balanced = non_onlygood_df.sample(min_count, random_state=42)

    # Combine and shuffle the balanced dataset
    balanced_df = pd.concat([onlygood_df_balanced, non_onlygood_df_balanced]).sample(frac=1, random_state=42).reset_index(drop=True)

    return balanced_df


In [ ]:
# balanced_df = get_balanced_onlygood_df(data_cleaned)
# balanced_df['Good'] = ((balanced_df['Wave Pattern'] == 1) | (balanced_df['Rivermouth'] == 1) | (balanced_df['Bay'] == 1)).astype(int)


In [ ]:
def count_and_sum_features(df):
    # Count the occurrences of each feature and the absence of all
    is_bay = (df['Bay'] == 1)
    is_rivermouth = (df['Rivermouth'] == 1)
    is_wavepattern = (df['Wave Pattern'] == 1)
    is_none = (~is_bay & ~is_rivermouth & ~is_wavepattern)  # None of the features are present

    # Summing the True values to get the count of rows meeting the conditions
    total_bay = is_bay.sum()
    total_rivermouth = is_rivermouth.sum()
    total_wavepattern = is_wavepattern.sum()
    total_none = is_none.sum()

    # Sum of instances where any of Bay, Rivermouth, or Wave Pattern is present
    total_feature_present = total_bay + total_rivermouth + total_wavepattern

    print(f"Total 'Bay' instances: {total_bay}")
    print(f"Total 'Rivermouth' instances: {total_rivermouth}")
    print(f"Total 'Wave Pattern' instances: {total_wavepattern}")
    print(f"Total 'None' instances: {total_none}")
    print(f"Total instances with any feature (Bay, Rivermouth, Wave Pattern): {total_feature_present}")
    print(f"Total instances with none of the features: {total_none}")

# Example usage, assuming balanced_onlygood_df is your DataFrame
# count_and_sum_features(balanced_df)


Total 'Bay' instances: 82
Total 'Rivermouth' instances: 63
Total 'Wave Pattern' instances: 89
Total 'None' instances: 170
Total instances with any feature (Bay, Rivermouth, Wave Pattern): 234
Total instances with none of the features: 170


In [ ]:
bay_df,rivermouth_df,wavepattern_df,other_df = get_df_by_feature(data_cleaned)
print(bay_df['Bay'].value_counts())
print(rivermouth_df['Rivermouth'].value_counts())
print(wavepattern_df['Wave Pattern'].value_counts())

In [ ]:
# print(bay_df['Bay'].value_counts())
# print(rivermouth_df['Rivermouth'].value_counts())
# print(wavepattern_df['Wave Pattern'].value_counts())

Bay
0    82
1    82
Name: count, dtype: int64
Rivermouth
1    63
0    63
Name: count, dtype: int64
Wave Pattern
1    89
0    89
Name: count, dtype: int64


#Data Uploading

In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
import os

def split_and_save_data(df, output_dir, stratify_by_cols, target_label):
    # Extend the stratification column to include the target label
    df['stratify_col'] = df[stratify_by_cols[0]] + "_" + df[stratify_by_cols[1]] + "_" + df[target_label].astype(str)
    counts = df['stratify_col'].value_counts()
    small_categories = counts[counts <5].index
    print(counts)
    # Assign a new category for small groups
    df['stratify_col_adjusted'] = df['stratify_col'].apply(lambda x: 'Other' if x in small_categories else x)
    adjusted_counts = df['stratify_col_adjusted'].value_counts()
    print(adjusted_counts)
    # Initialize the StratifiedShuffleSplit object
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)  # Split for test set 10%

    # Performing the first split to separate out the test set
    for train_val_index, test_index in sss.split(df, df['stratify_col_adjusted']):
        train_val_set = df.iloc[train_val_index]
        test_set = df.iloc[test_index]

    # Adjust test_size for validation set to be 20% of the remaining data
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)  # Split for validation set

    for train_index, val_index in sss.split(train_val_set, train_val_set['stratify_col_adjusted']):
        train_set = train_val_set.iloc[train_index]
        val_set = train_val_set.iloc[val_index]

    # Save datasets based on target label
    def save_datasets(dataset, set_name):
        target_dir = os.path.join(output_dir, set_name)
        os.makedirs(target_dir, exist_ok=True)

        # Save the entire dataset
        dataset.to_csv(os.path.join(target_dir, f'{set_name}_set.csv'), index=False)

    # Perform saving for train, validation, and test sets
    save_datasets(train_set, 'train')
    save_datasets(val_set, 'val')
    save_datasets(test_set, 'test')

    print(f"Data split into train, validation, and test sets and saved to {output_dir}")

# Example usage
# Assuming wavepattern_df is your DataFrame, 'Bay' is your target_label, and you have 'terrain' and 'tagger' as stratify columns
# split_and_save_data(wavepattern_df, 'path/to/output', ['terrain', 'tagger'], 'Bay')


In [ ]:
# split_and_save_data(wavepattern_df, '/content/drive/MyDrive/Satalite_Surf_Project/wavepattern_model', ['terrain', 'tagger'],'Wave Pattern')
# split_and_save_data(rivermouth_df, '/content/drive/MyDrive/Satalite_Surf_Project/rivermouth_model', ['terrain', 'tagger'],'Rivermouth')
# split_and_save_data(bay_df, '/content/drive/MyDrive/Satalite_Surf_Project/bay_model', ['terrain', 'tagger'],'Bay')
# split_and_save_data(other_df, '/content/drive/MyDrive/Satalite_Surf_Project/other_model', ['terrain', 'tagger'])
# split_and_save_data(other_df, '/content/drive/MyDrive/Satalite_Surf_Project/combined_model', ['terrain', 'tagger'])
# split_and_save_data(balanced_df, '/content/drive/MyDrive/Satalite_Surf_Project/alltogether_model', ['terrain', 'tagger'],'Good')


stratify_col
Unknown_Ori_0       34
Unknown_Gilad_0     34
Tropical_Alon_0     27
Desert_Alon_1       25
Unknown_Ori_1       24
Unknown_Eyal_1      24
Unknown_Gilad_1     20
Tropical_Gilad_1    19
Tropical_Alon_1     19
Desert_Alon_0       18
Tropical_Ron_0      15
Unknown_Ron_0       13
Tropical_Eyal_1     13
Tropical_Ron_1      12
Tropical_Gilad_0    12
Tropical_Ori_0      12
Unknown_Ron_1        8
Tropical_Ori_1       5
Unknown_Eyal_0       3
Desert_Ron_0         1
Desert_Gilad_1       1
Tropical_Eyal_0      1
Name: count, dtype: int64
stratify_col_adjusted
Unknown_Gilad_0     34
Unknown_Ori_0       34
Tropical_Alon_0     27
Desert_Alon_1       25
Unknown_Eyal_1      24
Unknown_Ori_1       24
Unknown_Gilad_1     20
Tropical_Alon_1     19
Tropical_Gilad_1    19
Desert_Alon_0       18
Tropical_Ron_0      15
Tropical_Eyal_1     13
Unknown_Ron_0       13
Tropical_Ron_1      12
Tropical_Gilad_0    12
Tropical_Ori_0      12
Unknown_Ron_1        8
Other                6
Tropical_Ori_1     

In [ ]:
import pandas as pd
import os
import shutil

def populate_images(csv_path, source_dir, target_dir, target_label):
    # Ensure the base target directory exists
    os.makedirs(target_dir, exist_ok=True)

    # Create subdirectories for target and non-target images
    target_images_dir = os.path.join(target_dir, f'{target_label}')
    non_target_images_dir = os.path.join(target_dir, f'not_{target_label}')
    os.makedirs(target_images_dir, exist_ok=True)
    os.makedirs(non_target_images_dir, exist_ok=True)

    # Read the CSV file
    df = pd.read_csv(csv_path)

    # Copy each image listed in the CSV to the appropriate target directory
    for index, row in df.iterrows():
        image_name = row['image_path']
        label = row[target_label]  # Assuming the CSV contains a column with boolean or 1/0 indicating target label
        source_path = os.path.join(source_dir, image_name)
        target_path = os.path.join(target_images_dir if label == 1 else non_target_images_dir, image_name)

        # Check if the source image exists before copying
        if os.path.exists(source_path):
            shutil.copy(source_path, target_path)
        else:
            print(f"Warning: {source_path} does not exist.")

def run_populate_images(model_name,target_label):
# Directory paths
  source_images_dir = '/content/images/Sat_Images'
  train_dir = f'/content/drive/MyDrive/Satalite_Surf_Project/{model_name}/train'
  validation_dir = f'/content/drive/MyDrive/Satalite_Surf_Project/{model_name}/val'
  test_dir = f'/content/drive/MyDrive/Satalite_Surf_Project/{model_name}/test'

  # Populate images for each set
  populate_images(f'/content/drive/MyDrive/Satalite_Surf_Project/{model_name}/train/train_set.csv', source_images_dir, train_dir,target_label)
  populate_images(f'/content/drive/MyDrive/Satalite_Surf_Project/{model_name}/val/val_set.csv', source_images_dir, validation_dir,target_label)
  populate_images(f'/content/drive/MyDrive/Satalite_Surf_Project/{model_name}/test/test_set.csv', source_images_dir, test_dir,target_label)

# Directory paths
# source_images_dir = '/content/images/Sat_Images'
# train_dir = '/content/drive/MyDrive/Satalite_Surf_Project/bay_model/train/images'
# validation_dir = '/content/drive/MyDrive/Satalite_Surf_Project/bay_model/val/images'
# test_dir = '/content/drive/MyDrive/Satalite_Surf_Project/bay_model/test/images'

# # Populate images for each set
# populate_images('/content/drive/MyDrive/Satalite_Surf_Project/bay_model/train/train_set.csv', source_images_dir, train_dir)
# populate_images('/content/drive/MyDrive/Satalite_Surf_Project/bay_model/val/val_set.csv', source_images_dir, validation_dir)
# populate_images('/content/drive/MyDrive/Satalite_Surf_Project/bay_model/test/test_set.csv', source_images_dir, test_dir)


In [ ]:
# zip_path = '/content/drive/MyDrive/Satalite_Surf_Project/Sat_Images.zip'
# unzipped_images_dir = '/content/images'
# unzip_images(zip_path,unzipped_images_dir)

Images extracted to /content/images


In [ ]:
# run_populate_images('rivermouth_model','Rivermouth')
# run_populate_images('wavepattern_model','Wave Pattern')
# run_populate_images('bay_model','Bay')
# run_populate_images('alltogether_model','Good')